In [0]:
CREATE OR REPLACE STREAMING TABLE bronze_material_master
TBLPROPERTIES(
  'delta.enableChangeDataFeed' = 'true',
  "checkpointLocation" = "/Volumes/workspace/spl/datastore/checkpoint/material_master",
  'delta.columnMapping.mode' = 'name'
)
AS
SELECT 
  CAST(material_id     AS STRING) AS material_id,
  CAST(material_name   AS STRING) AS material_name,
  CAST(category        AS STRING) AS category,
  CAST(sub_category    AS STRING) AS sub_category,
  CAST(uom             AS STRING) AS uom,
  CAST(unit_cost       AS STRING) AS unit_cost,
  CAST(supplier_name   AS STRING) AS supplier_name,
  CAST(country         AS STRING) AS country,
  CAST(plant           AS STRING) AS plant,
  CAST(status          AS STRING) AS status,
  CAST(last_updated    AS STRING) AS last_updated,
  CAST(lead_time_days  AS STRING) AS lead_time_days,
  CAST(safety_stock    AS STRING) AS safety_stock,
  CAST(reorder_level   AS STRING) AS reorder_level,
  CAST(remarks         AS STRING) AS remarks,
  current_timestamp() as load_dt,
  _metadata.file_path as source_file_path,
  _metadata.file_name as source_file_name
FROM STREAM cloud_files(
  '/Volumes/workspace/material_master_dbc_10/datastore/', 
  'csv',
  map(
    'delimiter', '|',
    'header', 'true',
    'multiLine', 'false',
    'cloudFiles.inferColumnTypes', 'true',
    'cloudFiles.schemaLocation', '/Volumes/workspace/material_master_dbc_10/datastore/material_master_1k.csv'
  )
)

In [0]:
CREATE OR REFRESH LIVE TABLE clean_material_master_silver
TBLPROPERTIES ("quality" = "silver")
CONSTRAINT material_id_not_null EXPECT (material_id IS NOT NULL) ON VIOLATION DROP ROW
AS
WITH raw AS (
  SELECT
    TRIM(material_id)        AS material_id,
    TRIM(material_name)      AS material_name,
    TRIM(category)           AS category,
    TRIM(sub_category)       AS sub_category,
    TRIM(uom)                AS uom,
    TRIM(unit_cost)          AS unit_cost_str,
    TRIM(supplier_name)      AS supplier_name,
    TRIM(country)            AS country,
    TRIM(plant)              AS plant,
    TRIM(status)             AS status,
    TRIM(last_updated)       AS last_updated_str,
    TRIM(lead_time_days)     AS lead_time_days_str,
    TRIM(safety_stock)       AS safety_stock_str,
    TRIM(reorder_level)      AS reorder_level_str,
    NULLIF(TRIM(remarks),'') AS remarks,
    load_dt
  FROM STREAM(LIVE.bronze_material_master)   
),
typed AS (
  SELECT
    NULLIF(material_id,      '')                      AS material_id,
    NULLIF(material_name,    '')                      AS material_name,
    NULLIF(category,         '')                      AS category,
    NULLIF(sub_category,     '')                      AS sub_category,
    NULLIF(uom,              '')                      AS uom,
    TRY_CAST(unit_cost_str       AS DECIMAL(10,2))    AS unit_cost,
    NULLIF(supplier_name,    '')                      AS supplier_name,
    NULLIF(country,          '')                      AS country,
    NULLIF(plant,            '')                      AS plant,
    NULLIF(status,           '')                      AS status,
    TRY_CAST(last_updated_str    AS DATE)             AS last_updated,
    TRY_CAST(lead_time_days_str  AS INT)              AS lead_time_days,
    TRY_CAST(safety_stock_str    AS INT)              AS safety_stock,
    TRY_CAST(reorder_level_str   AS INT)              AS reorder_level,
    remarks,
    COALESCE(load_dt, current_timestamp())            AS load_dt
  FROM raw
)
SELECT *
FROM typed
WHERE unit_cost   IS NOT NULL
  AND last_updated IS NOT NULL;